In [5]:
import pandas as pd
import numpy as np
from ucimlrepo import fetch_ucirepo
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import joblib

In [6]:

dataset = fetch_ucirepo(id=235)
df = pd.concat([dataset.data.features, dataset.data.targets], axis=1)
df.replace('?', np.nan, inplace=True)
df.dropna(inplace=True)

# Convert numeric columns
for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors='ignore')

# Convert datetime
df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
df['hour'] = df['DateTime'].dt.hour
df['weekday'] = df['DateTime'].dt.weekday

# Drop unnecessary columns
df.drop(['Date', 'Time', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3'], axis=1, inplace=True)


c:\Users\104875\AppData\Local\Programs\Python\Python310\lib\site-packages\ucimlrepo\fetch.py:97: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_url)
C:\Users\104875\AppData\Local\Temp\ipykernel_6768\1668014526.py:8: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_numeric(df[col], errors='ignore')
C:\Users\104875\AppData\Local\Temp\ipykernel_6768\1668014526.py:11: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])


In [7]:

# Reduce memory usage
for col in df.select_dtypes(include=['float']):
    df[col] = pd.to_numeric(df[col], downcast='float')
for col in df.select_dtypes(include=['int']):
    df[col] = pd.to_numeric(df[col], downcast='integer')


In [8]:

features = ['hour', 'weekday', 'Voltage', 'Global_intensity']
target = 'Global_active_power'

X = df[features]
y = df[target]

# Sampling to reduce size
X_sample, _, y_sample, _ = train_test_split(X, y, train_size=0.3, random_state=42)

# Train Random Forest (simplified)
model = RandomForestRegressor(n_estimators=10, max_depth=5, random_state=42)
model.fit(X_sample, y_sample)

# Save model (compressed)
joblib.dump(model, 'Ops_rf_model.pkl', compress=3)


['Ops_rf_model.pkl']

In [10]:
import os

# Save model (compressed) to an older location as well
joblib.dump(model, 'Ops_rf_model.pkl', compress=3)

# Ensure /tmp directory exists before saving
os.makedirs('/tmp', exist_ok=True)
joblib.dump(model, '/tmp/Ops_rf_model_old.pkl', compress=3)

['/tmp/Ops_rf_model_old.pkl']